# CLIP Greenhouse Gas Inversions

Paper source: https://essd.copernicus.org/articles/14/1639/2022/ 

Data Source: https://zenodo.org/record/5089799#.ZGTewexByAl (data compressed into a single zip file)

## Setup

In [129]:
# Import necessary libraries
import os
import zipfile
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import altair as alt

## Data Preparation
Place datasets in same folder as notebook to proceed.

In [130]:
# Define the paths for the zip file and extraction
zip_path = "climate_inversion_data.zip"
extract_path = "data/climate_inversion_data"

# Create a new directory to extract the files into
os.makedirs(extract_path, exist_ok=True)

# Open the zip file and extract all files
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# List the files in the directory
file_list = os.listdir(extract_path)

# Define the paths to the CSV files
inventory_path = os.path.join(extract_path, "Inventory_1990-2019.csv")
co2_inversion_path = os.path.join(extract_path, "CO2_inversion_1990-2019 (1).csv")

# Load the CSV files into dataframes
inventory_df = pd.read_csv(inventory_path)
co2_inversion_df = pd.read_csv(co2_inversion_path)

# rename column names (lower case, remove whitespaces)
inventory_df.columns= inventory_df.columns.str.strip().str.lower().str.replace(' ','_')
co2_inversion_df.columns= co2_inversion_df.columns.str.strip().str.lower().str.replace(' ','_')

# Replace EUA with EUR for correct match of both datasets
inventory_df['party'] = inventory_df['party'].str.replace('EUA','EUR')

In [131]:
inventory_df.head()

,party,year,gas,sector,value,unit
0,ARE,1994,CH4,agriculture & waste,0.1567,Tg CH4
1,ARE,1994,CH4,anthropogenic,0.5537,Tg CH4
2,ARE,1994,CH4,fossil,0.3970,Tg CH4
3,ARE,2000,CH4,agriculture & waste,0.2010,Tg CH4
4,ARE,2000,CH4,anthropogenic,0.9980,Tg CH4


In [132]:
co2_inversion_df.head()

,party,year,sector,cams,carboscope,cte,miroc,nismon,uoe
0,CHN,1990,land flux (all land),-193.650,-381.48,NaN,NaN,-401.1000,NaN
1,CHN,1991,land flux (all land),68.593,-312.91,NaN,NaN,-402.2800,NaN
2,CHN,1992,land flux (all land),-300.620,-310.75,NaN,NaN,-213.7100,NaN
3,CHN,1993,land flux (all land),-252.990,-290.99,NaN,NaN,-516.2700,NaN
4,CHN,1994,land flux (all land),26.949,-241.00,NaN,NaN,2.8735,NaN


In [133]:
# List of countries to include in the dataframe (Optional)
countries = ["CHN", "USA", "EUR", "RUS", "CAN", "KAZ", "MNG", "IND", "BRA", "COD", "ZAF", "AUS"]

# Filter the inversion data for the specified countries
co2_inversion_df_filtered = co2_inversion_df[co2_inversion_df["party"].isin(countries)]

# Calculate the median and range of inversions per year for each country
co2_inversion_df_filtered["inversion_median"] = co2_inversion_df_filtered[["cams", "carboscope", "cte", "miroc", "nismon", "uoe"]].median(axis=1)
co2_inversion_df_filtered["inversion_min"] = co2_inversion_df_filtered[["cams", "carboscope", "cte", "miroc", "nismon", "uoe"]].min(axis=1)
co2_inversion_df_filtered["inversion_max"] = co2_inversion_df_filtered[["cams", "carboscope", "cte", "miroc", "nismon", "uoe"]].max(axis=1)

# Filter the inventory data for the specified countries and for CO2
inventory_df_filtered = inventory_df[(inventory_df["party"].isin(countries)) & (inventory_df["gas"] == "CO2")]

# Group the inventory data by country and year, and calculate the sum of CO2 emissions
inventory_df_grouped = inventory_df_filtered.groupby(["party", "year"])["value"].sum().reset_index()

# Merge the two dataframes on "Party" and "Year"
merged_df = pd.merge(co2_inversion_df_filtered, inventory_df_grouped, on=["party", "year"], how="outer")

# Rename the "Value" column to "National Inventory"
merged_df.rename(columns={"value": "national_inventory"}, inplace=True)

# Filter the data to only include "land flux (all land)"
land_flux_df = merged_df[merged_df["sector"] == "land flux (all land)"]

land_flux_df.head()

<ipython-input-133-dd648c450ce2>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  co2_inversion_df_filtered["inversion_median"] = co2_inversion_df_filtered[["cams", "carboscope", "cte", "miroc", "nismon", "uoe"]].median(axis=1)
<ipython-input-133-dd648c450ce2>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  co2_inversion_df_filtered["inversion_min"] = co2_inversion_df_filtered[["cams", "carboscope", "cte", "miroc", "nismon", "uoe"]].min(axis=1)
<ipython-input-133-dd648c450ce2>:10: SettingWithCopyWarning: 


,party,year,sector,cams,carboscope,cte,miroc,nismon,uoe,inversion_median,inversion_min,inversion_max,national_inventory
0,CHN,1990,land flux (all land),-193.650,-381.48,NaN,NaN,-401.1000,NaN,-381.4800,-401.10,-193.650,NaN
3,CHN,1991,land flux (all land),68.593,-312.91,NaN,NaN,-402.2800,NaN,-312.9100,-402.28,68.593,NaN
6,CHN,1992,land flux (all land),-300.620,-310.75,NaN,NaN,-213.7100,NaN,-300.6200,-310.75,-213.710,NaN
9,CHN,1993,land flux (all land),-252.990,-290.99,NaN,NaN,-516.2700,NaN,-290.9900,-516.27,-252.990,NaN
12,CHN,1994,land flux (all land),26.949,-241.00,NaN,NaN,2.8735,NaN,2.8735,-241.00,26.949,-111.130636


## Data Visualization

### Exploratory chart: not for use as little picture

In [144]:
df_viz = land_flux_df[['party', 'year', 'inversion_median', 'inversion_min', 'inversion_max', 'national_inventory']].copy()
#df_viz = df_viz.fillna(0)
df_viz['difference'] = df_viz['inversion_median'] - df_viz['national_inventory']

countries = ['CAN','RUS','USA','EUR','CHN','IND','COD','BRA','AUS']
df_viz = df_viz.loc[df_viz['party'].isin(countries)]

df_viz.head(20)

,party,year,inversion_median,inversion_min,inversion_max,national_inventory,difference
0,CHN,1990,-381.4800,-401.10,-193.650,NaN,NaN
3,CHN,1991,-312.9100,-402.28,68.593,NaN,NaN
6,CHN,1992,-300.6200,-310.75,-213.710,NaN,NaN
9,CHN,1993,-290.9900,-516.27,-252.990,NaN,NaN
12,CHN,1994,2.8735,-241.00,26.949,-111.130636,114.004136
15,CHN,1995,-138.5100,-327.72,-124.330,NaN,NaN
18,CHN,1996,-296.5700,-322.84,-53.868,NaN,NaN
21,CHN,1997,-285.2700,-641.50,-99.208,NaN,NaN
24,CHN,1998,-125.3820,-338.50,475.020,NaN,NaN
27,CHN,1999,-176.9055,-333.04,281.150,NaN,NaN


In [145]:
# Convert 'party' column to string
df_viz['party'] = df_viz['party'].astype(str)

chart = alt.Chart(df_viz).mark_rect().encode(
    x=alt.X('year:O', axis=None),
    y=alt.Y('party:N', sort=countries, axis=None),
    color=alt.Color('difference:Q', scale=alt.Scale(scheme='pinkyellowgreen', reverse=True), legend=None)
)

text = chart.mark_text(baseline='middle', font='Helvetica', fontSize=14).encode(
    text=alt.condition(
        alt.datum.year == 2004,  # Label only for year 2004
        alt.Text('party:N'),
        alt.value('')
    ),
    color=alt.value('black')  # Set the default text color to black
)

layer = alt.layer(chart, text).configure_axis(labels=False, ticks=False)
layer = layer.configure(background='darkgrey').properties(width=600, height=600)

layer

alt.LayerChart(...)

### Individual Charts per Country


In [158]:
# copy relevant columns to new df
df_viz = land_flux_df[['party', 'year', 'inversion_median', 'inversion_min', 'inversion_max', 'national_inventory']].copy()

# filter party / region
df_viz = df_viz.loc[df_viz['party'] == 'BRA'] 

# create difference column between inv. median and national_inventory
df_viz['difference'] = df_viz['inversion_median'] - df_viz['national_inventory']

df_viz.head()

,party,year,inversion_median,inversion_min,inversion_max,national_inventory,difference
768,BRA,1990,190.9100,49.6430,307.7600,237.599182,-46.689182
771,BRA,1991,-108.3200,-262.1900,293.0600,189.400636,-297.720636
774,BRA,1992,-243.9000,-300.3400,255.8300,217.833000,-461.733000
777,BRA,1993,294.9000,-101.3000,314.8000,227.811545,67.088454
780,BRA,1994,463.2300,-103.4900,740.1200,221.842909,241.387091
783,BRA,1995,69.0980,-73.9370,493.6600,513.502909,-444.404909
786,BRA,1996,-106.9645,-387.8900,129.8500,324.496909,-431.461409
789,BRA,1997,241.8900,-7.8020,293.1400,237.191727,4.698273
792,BRA,1998,493.5400,-30.6560,1036.5000,304.746273,188.793727
795,BRA,1999,26.4415,-9.3867,67.6970,292.200818,-265.759318


#### Recreation of Charts from Paper

As a first element the charts from the paper are recreated, albeit in a more stylised form.

In [156]:
# create chart
area = alt.Chart(df_viz).mark_area(color='#c36b95', opacity=1, interpolate='cardinal').encode(
    x = alt.X('year', title=None),
    y = alt.Y('inversion_min:Q', title=None),
    y2 = alt.Y2('inversion_max:Q', title=None)
)

line = alt.Chart(df_viz).mark_trail(color='#579cc5', opacity=1, interpolate='cardinal').encode(
    x = alt.X('year', title=None, axis=None),
    y = alt.Y('inversion_median:Q', title=None, axis=None),
    size=alt.Size('inversion_median:Q', legend=None)
)

points = alt.Chart(df_viz).mark_circle(color='#fff').encode(
    x = alt.X('year', title=None, axis=None),
    y = alt.Y('national_inventory:Q', title=None, axis=None)
)

chart = alt.layer(area, line, points).properties(
    width=600,
    height=600
).configure(
    padding={"left": 40, "top": 40, "right": 40, "bottom": 40},
    background='#6BC0C8'
).configure_view(
    strokeWidth=0
).configure_axis(
    grid=False
)

chart

alt.LayerChart(...)

### Abstraction: Only showing difference between reported and measured data
In the spirit of creating little pictures the chart is broken down to the simple message "what is the delta between observed and reported CO2 emissions?"
The resulting chart is meant to be further processed in vector editing tools.

In [155]:
# create chart
chart = alt.Chart(df_viz).mark_bar(opacity=1, size=15).encode(
    x = alt.X('year', title=None, axis=None),
    y = alt.Y('difference:Q', title=None, axis=None),
    color=alt.condition(
        alt.datum.difference > 0,
        alt.value("#D5EAD9"),  # positive color
        alt.value("#6E4A2D")  # negative color
    ),
    tooltip=['year', 'difference']
).properties(
    width=600,
    height=600
).configure(
    padding={"left": 0, "top": 0, "right": 0, "bottom": 0},
    background='#6BC0C8'
).configure_view(
    strokeWidth=0
).configure_axis(
    grid=False
)

chart

alt.Chart(...)